In [1]:
import cv2
import mediapipe as mp
import time

import math

import numpy as np
from mxnet import nd

# Mediapipe를 이용하여 FPS, Velocity, Angle을 realtime video에 나타내기

In [7]:
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

def dotproduct(v1, v2):
  d = nd.dot(v1, v2)
  return d.asscalar()

def length(v):
  return math.sqrt(dotproduct(v, v))


def angle(v1, v2):
  if (length(v1) * length(v2)) != 0 and -1 <= dotproduct(v1, v2) / (length(v1) * length(v2)) and  dotproduct(v1, v2) / (length(v1) * length(v2))<= 1:
      a = math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))
  else:
      a = 0
  return np.degrees(a)


#capture = cv2.VideoCapture("D:/simple_Pose/videos/golf_swing4.mp4")
#capture = cv2.VideoCapture("D:/simple_Pose/videos/test.mp4")

capture = cv2.VideoCapture(0,cv2.CAP_MSMF)
capture.set(cv2.CAP_PROP_FPS,30)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)



# used to record the time when we processed last frame
prev_frame_time = 0
 
# used to record the time at which we processed current frame
new_frame_time = 0


prev_right_wrist_x = 0
prev_right_wrist_y = 0


right_wrist_deg = 0
right_shoulder_deg = 0
right_elbow_deg = 0
right_thumb_deg = 0
right_index_deg = 0
right_pinky_deg = 0

count_check = 0

while True:
    sucess, img = capture.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = pose.process(imgRGB)
    #print(result.pose_world_landmarks)
    #mpDraw.draw_landmarks(img,result.pose_world_landmarks,mpPose.POSE_CONNECTIONS)
    
    if result.pose_landmarks:
        mpDraw.draw_landmarks(img, result.pose_landmarks,mpPose.POSE_CONNECTIONS)
        for id, lm in enumerate((result.pose_landmarks.landmark)):
            h,w,c = img.shape
#             print(id,lm)
            cx,cy=int(lm.x*w), int(lm.y*h)
           # cv2.circle(img,(cx,cy),10,(255,0,0),cv2.FILLED)
            if id == 12:
                 right_shoulder_x = lm.x 
                 right_shoulder_y = lm.y
            elif id == 24:
                right_hip_x = lm.x
                right_hip_y = lm.y
                count_check = 1
            elif id == 14:
                right_elbow_x = lm.x
                right_elbow_y = lm.y
            elif id == 16:
                right_wrist_x = lm.x
                right_wrist_y = lm.y
            elif id == 22:          # 엄지
                right_thumb_x = lm.x
                right_thumb_y = lm.y
            elif id == 20:          # 검지     
                right_index_x = lm.x
                right_index_y = lm.y    
            elif id == 18:          # 약지
                right_pinky_x = lm.x
                right_pinky_y = lm.y
            if count_check == 1:
                right_wrist_deg = (np.abs(math.atan2(right_thumb_y-right_wrist_y, right_thumb_x-right_wrist_x) - math.atan2(right_wrist_y-right_elbow_y, right_wrist_x - right_elbow_x))*180)/math.pi
                right_shoulder_deg = (np.abs(math.atan2(right_hip_y-right_shoulder_y, right_hip_x-right_shoulder_x) - math.atan2(right_elbow_y-rigrh_shoulder_y, right_elbow_x - rigrh_shoulder_x))*180)/math.pi
                right_elbow_deg = (np.abs(math.atan2(right_shoulder_y-right_elbow_y, right_shoulder_x-right_elbow_x) - math.atan2(right_wrist_y-right_elbow_y, right_wrist_x-right_elbow_x))*180)/math.pi
                right_thumb_deg = (np.abs(math.atan2(right_thumb_y-right_wrist_y, right_thumb_x-right_wrist_x) - math.atan2(right_thumb_y-right_index_y, right_thumb_x-right_index_x))*180)/math.pi
                right_index_deg = (np.abs(math.atan2(right_index_y-right_wrist_y, right_index_x-right_wrist_x) - math.atan2(right_index_y-right_pinky_y, right_index_x-right_pinky_x))*180)/math.pi
                right_pinky_deg = (np.abs(math.atan2(right_pinky_y-right_wrist_y, right_pinky_x-right_wrist_x) - math.atan2(right_pinky_y-right_index_y, right_pinky_x-right_index_x))*180)/math.pi
                count_check = 0       

            
    prev_frame_coor = np.array([prev_right_wrist_x, prev_right_wrist_y])
    new_frame_coor = np.array([right_wrist_x, right_wrist_y])
    
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    fps1 = round(1/fps, 2)  # 한 프레임 당 초
    v = np.sqrt(np.sum(np.square(prev_frame_coor - new_frame_coor)))/fps1
    print(v*20)
    
    
    prev_frame_time = new_frame_time
    prev_right_wrist_x = right_wrist_x
    prev_right_wrist_y = right_wrist_y
    
    cv2.putText(img, "fps:" + str(int(fps)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)shoulder degree:" + str(int(right_shoulder_deg)), (70,100), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)elbow degree:" + str(int(right_elbow_deg)), (70,150), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)    
    cv2.putText(img, "(r)wrist degree:" + str(int(right_wrist_deg)), (70,200), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)thumb degree:" + str(int(right_thumb_deg)), (70,250), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)index degree:" + str(int(right_index_deg)), (70,300), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)pinky degree:" + str(int(right_pinky_deg)), (70,350), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    # 속도 변화를 더 잘 보이게 하기 위해 *20 해줌
    cv2.putText(img, "(r)wrist velocity:" + str(v*20), (70,400), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
capture.release()
cv2.destroyAllWindows()

1.128329400710618e-08
69.89778713216567
36.95653333566464
60.5491434825759
52.3606479838977
8.345556531696896
13.291103573126
10.445118002905938
8.647301730586873
114.65026313098092
115.86925761439588
100.19698355481998
41.68931708711925
192.58951562514736
19.741437687338397
53.33534614019513
23.765426727104995
2.458483295359835
8.377370710841122
35.32334648870545
8.422168786937206
7.116295306075205
52.154482737894384
104.43075524535942
21.45642638955776
18.897073246508025
50.62647126406661
13.105421172654466
21.520686949828335
15.030887958078093
5.81821569427332
13.056075773146665
46.03290438336253
97.97105211547951
9.537948665044548
20.545928713466832
95.34045632269087
6.004647690312108
44.74830241042498
33.83668911738012
59.36403069852183
11.92528883962863
35.30022857444941
26.62664202528932
7.907207633073857
90.46144811489258
48.11949023162812
38.639312278198624
2.6306806332403863
43.92161248498378
14.390504394400825
29.20399722507621
23.033007720512444
11.83741151353023
28.0695372